<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1602, done.
remote: Counting objects: 100% (1602/1602), done.
remote: Compressing objects: 100% (1266/1266), done.
remote: Total 1602 (delta 1041), reused 627 (delta 319), pack-reused 0
Receiving objects: 100% (1602/1602), 88.32 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (1041/1041), done.
/content/teambrainiac/source/DL


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [4]:
# Possible Missing Packages
!pip install boto3
!pip install nilearn

     |████████████████████████████████| 132 kB 4.1 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 8.7 MB 40.5 MB/s 
     |████████████████████████████████| 138 kB 59.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 9.6 MB 3.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 127 kB 47.5 MB/s 
  Attempting uninstall: urllib3
    Found existing inst

In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import TensorDataset

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader


## Import Group fMRI Data from AWS

In [ ]:
%%time
path = 'dl/partition_train_2.pkl'
train_images = access_load_data(path, False)

In [8]:
train_images['images'].shape

torch.Size([1512, 1, 79, 95, 79])

## Build Test Batch
Use this code only if you are changing the model based on a subset of the data

In [14]:
# train_batch_size = 18

# x = train_images['images'][:train_batch_size]
# y = train_images['labels'][:train_batch_size]

## Build Dataloader

In [15]:
# May move this into the training bit
# bs = 150

# x_train = train_images['images']
# y_train = train_images['labels']

# ds = TensorDataset(x_train, y_train)
# dl = DataLoader(ds, batch_size = bs, shuffle=True)

## Build Model

In [10]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     

  def forward(self, xb):
    xb = self.bn1(F.relu(self.conv1((xb))))
    xb = self.bn2(F.relu(self.conv2((xb)))) # Takes a long time
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.bn4(F.relu(self.conv4((xb))))
    xb = self.pool1(xb)
    xb = xb.view(xb.shape[:2])
    xb = self.fc1(xb)
    xb = self.fc2(xb)
    return xb      

    

In [11]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
# print("First model training on GPU")
# print(model)

# Initialize other parameters
epochs = 10
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

In [12]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()



# Use to load GPU model
# model.load_state_dict(torch.load(path))



In [13]:

def run_cnn(model, epochs, learning_rate, loss_func, opt, dl):
  metrics_dict = {}
  # Run Model
  for epoch in range(1, 1+epochs):
    i = 1
    accuracy_list = []
    loss_list = []
    model.train()
    print('epoch', epoch)
    for xb, yb in dl:
      print('batch', i)
      i += 1

      xb = xb.float()
      pred = model(xb)
      loss_batch = loss_func(pred, yb)
      loss_list.append(loss_batch)
      accuracy_batch = accuracy(pred, yb)
      accuracy_list.append(accuracy_batch)

      print('Batch Loss', loss_batch)
      print('Batch Accuracy', accuracy_batch)

      loss_batch.backward()
      opt.step()
      opt.zero_grad()

    model.eval()
    
    print('Saving model')
    model_name = 'cnn_fmri_initial_model.pt'
    path = F"/content/gdrive/My Drive/{model_name}" 
    torch.save(model.state_dict(), path)

    metrics_dict['epoch_'+str(epoch)] = {'accuracy':accuracy_list, 'loss':loss_list}

    print('epoch', epoch, 'finished\n')
    try:
      past_epoch_accuracies = [sum(metrics_dict['epoch_'+str(epoch-2)]['accuracy']), sum(metrics_dict['epoch_'+str(epoch-1)]['accuracy'])]
      current_epoch_accuracy = sum(metrics_dict['epoch_'+str(epoch)]['accuracy'])
      if past_epoch_accuracies[0] > current_epoch_accuracy and past_epoch_accuracies[1] > current_epoch_accuracy:
        print('Early stop to avoid overfitting\nModel accuracies did not decrease for two epochs')
        return model, metrics_dict

    except:
      pass
  
  return model, metrics_dict
  


In [19]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
# print("First model training on GPU")
# print(model)

# Initialize other parameters
epochs = 10
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum



In [20]:

metrics_dict = {}
for i,image_index in enumerate(range(0, train_images['images'].shape[0], 504)):

  bs = 36

  x_train = train_images['images'][image_index:image_index+504]
  y_train = train_images['labels'][image_index:image_index+504]

  ds = TensorDataset(x_train, y_train)
  dl = DataLoader(ds, batch_size = bs, shuffle=True)

  model, metrics = run_cnn(model, epochs, learning_rate, loss_func, opt, dl)
  metrics_dict['round_'+str(i)] = metrics
  
  f = open("/content/gdrive/My Drive/metrics_dict_%s.pkl"%('train_1'),"wb")
  pickle.dump(metrics_dict,f)
  print('Finshed with set', str(i), 'of 504 images\nStarting next set.\n\n')

epoch 1
batch 1
Batch Loss tensor(0.6929, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5278)
batch 2
Batch Loss tensor(0.6831, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.4722)
batch 3
Batch Loss tensor(0.8528, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5556)
batch 4
Batch Loss tensor(0.7260, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.4722)
batch 5
Batch Loss tensor(0.7298, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5833)
batch 6
Batch Loss tensor(0.7134, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.4722)
batch 7
Batch Loss tensor(0.6989, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.4722)
batch 8
Batch Loss tensor(0.7069, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.4722)
batch 9
Batch Loss tensor(0.6709, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5833)
batch 10
Batch Loss tensor(0.6858, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5278)
batch 11
Batch Loss tensor(0.7153, grad_fn=<NllLossBackward0>)
Batch Ac

## Training

In [ ]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_test_model(epochs, model, opt, train_dl, test_dl, device):
  for epoch in range(1, epochs+1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for xb, yb in tqdm(train_dl):
      xb = xb.to(device)
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      opt.zero_grad()
    
      # Forward pass through NN
      y_pred = model(xb)#.to(float)
      train_loss = criterion(y_pred, yb)
      train_acc = accuracy(y_pred, yb)

      # Backward pass, updating weights
      train_loss.backward()
      opt.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




# def accuracy(y_pred, y_test):
#   # Calculating model accuracy at each epoch 
#   y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
#   _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
#   correct_pred = (y_pred_prob == y_test).float()
#   acc = correct_pred.sum() / len(correct_pred)
#   acc = torch.round(acc * 100)

#   return acc



     





# if __name__ == '__main__':
#   train_val_model(epochs)

{'train': [], 'val': []}
{'train': [], 'val': []}

Epoch$ : 1


NameError: ignored